# ⚡ Microgrids simulator showcase: “Power for an island” ⚡

!["Power for an island" banner](images/Island_power_banner.svg)

Demo of the operational & economic simulation of an isolated microgrid (using [Microgrids.py](https://github.com/Microgrids-X/Microgrids.py)).

## Context: 

- Twenty kilometers off the coast of mainland France lies an island of 1,000 inhabitants with no connection to the mainland's power grid.
- This interactive demo allows you to investigate several combinations of energy sources to **supply this island with power**, reliably.
- Available power sources: 
    - solar panels and wind turbines
    - a dispatchable generator
    - a battery (to better balance the load and the available solar power)

**Caveat**: although we tried to put reasonable technical and economic data in this demo, do not take its outcome as a *“perfect scientific truth”* (if such a concept exists) of what should be deployed on this island and at which exact cost. However, this demo can show the positive impact of mixing solar and wind resources. 

## Instructions:

1. Use the Run button (▶️) or `Shift+Enter` to run selected “code cells” (gray background).
2. Run first the two code cells below to setup the demo
3. The real experiment starts when running the third cell (which contains `interactive(...)`)

**Remark**: although this app is displayed in a web browser, all data is stored and processed *locally on your computer* (unlike typical cloud apps) thanks to [JupyterLite](https://jupyterlite.readthedocs.io/en/latest/). This means that your privacy is respected: no record of your action is send to remote servers. Consequence: if you want a keep record of your experiments, use "File/Download".

In [1]:
try: # Install microgrids package in JupyterLite, if run in JupyterLite
    import piplite
    await piplite.install(['microgrids', 'ipywidgets'])
except ImportError:
    print('When run outside JupyterLite, this demo requires you to install the necessary packages by yourself')

When run outside JupyterLite, this demo requires you to install the necessary packages by yourself


In [16]:
%run -i microgrid_showcase_setup.py

Showcase setup complete.


## Experiment: interactive energy mix

This experiment starts from zero Solar power, Wind power and zero Battery energy, only using a large enough fuel-based Generator:

1. Start by first increasing Solar or Wind power alone
    - at first this reduce the usage of the Generator (see energy mix)
    - but beyond ~2000 kW (for Solar) or 1500 kW (for Wind), there is more and more *spilled energy*
2. Then, using 2000 kW of Solar and 2000 kW of Wind, increase Battery capacity to reduce spilled energy
    - at first, this reduces spilled energy and therefore reduces further generator usage
    - but beyond ~5000 kWh, it requires a higher and higher capacity to get an effect

With appropriate settings, you should find a Levelized Cost of Electricity below 0.22 \\$/kWh (while it is 0.35 \\$/kWh without PV and battery).
- You can also get a quite low LCOE with Wind alone or Solar alone (along with Generator and Storage).
  However, it's not as low as when Wind and Solar are used together → *Solar and Wind are very complementary*.

In [8]:
interactive(interactive_energy_mix,
            Solar=(0.0, 6000, 500),
            Wind=(0.0, 3000, 250),
            Battery=(0.0, 9000, 500))

interactive(children=(FloatSlider(value=0.0, description='Solar', max=6000.0, step=500.0), FloatSlider(value=0…

### Bonus: undersized generator – degraded quality of service

You can experiment with an *undersized* dispatchable generator (undersized = less than max load which is 1707 kW).
This yields some *“load shedding”*, that is a *degraded quality of service* where the desired electricity consumption is not satisfied at all times.

Then you can witness how the solar plant together with the battery can *improve* this quality of service (only partially though).

If you use generator which is only *slightly* undersize (1600 kW), you can even reach zero load shedding while saving a bit of money (LCOE below 0.21 \$/kWh)

**Intructions**: 
1. Run the cell to change the value of `power_rated_gen`. This is a global variable which sets the size of the generator.
2. Go back to the interactive plot above and move the slider to update the computation with the new generator size

In [7]:
power_rated_gen = 1000.  # < 1000 to see some significant load shedding (kW)

## Microgrid sizing optimization

DRAFT, to be updated once the notebook is finished. perhaps also add a button to run the process?

In [17]:
shed_max = 0.0
#xmin[0] = 1.800 # fix generator
#xmax[0] = 1.801

xopt, res, numevals = optim_mg(shed_max, algo='DIRECT', maxeval=300)
lcoe_opt, shed_rate_opt = obj_multi(xopt)
print(f'Recommended sizing after {numevals} calls to the microgrid simulator:')
print_xopt(xopt)
print(f'LCOE*: {lcoe_opt}', )
print(f'shed*: {shed_rate_opt}')

Recommended sizing after 307 calls to the microgrid simulator:
- Generator power:  1594 kW
- Storage capacity: 3805 kWh
- Solar power:      1980 kW
- Wind power:       1411 kW
LCOE*: 0.20886002338931575
shed*: 0.0


---
## Going further

To get more understanding of the [Microgrids.py](https://github.com/Microgrids-X/Microgrids.py) package
used for estimating the operation & economics of a microgrid project,
open the [Microgrid_Wind-Solar.ipynb](Microgrid_Wind-Solar.ipynb) notebook.
It contains the same example as here, but presented step-by-step instead of being packed inside the [microgrid_showcase_setup.py](microgrid_showcase_setup.py) script (which is run at the start).
This notebook also gives more explanations on how this example is formulated (data sources, parameters values…).

If you want to understand more about the sizing optimization, see the [Microgrid_sizing_optimization.ipynb](Microgrid_sizing_optimization.ipynb) notebook. It also uses the same example as here.

*Image credit*:
the “Power for an island” banner by Pierre Haessig is derived from the photo [Ile de Sein depuis la pointe du Raz](
https://www.flickr.com/photos/philippeos/10036523006/) by [philippematon](https://www.flickr.com/photos/philippeos/) on Flickr, 2013, [CC-BY](https://creativecommons.org/licenses/by/2.0/).